
## This notebook is for analysing VAAs - Volcanic Ash Advisories reports## 

Information   issued by a   Volcanic   Ash   Advisory   Center (VAAC)  concerning  the occurrence or expected  occurrence of volcanic  ash  that  may  affect the safety of aircraft operations. A VAA is a text message that identifies the volcano, time of eruption, observed position of the ash cloud, and the forecasted position of the ash. The
VAA is not to be used as a warning message. 

In the file *vaac_total_line.json* we can find information from many VAAs emited by different VAACs ( one entry by report).  We are going to a _DataFame_ to load the data and get the schema.

And we can explore/analyse different properties of the dataset:
    * Visualize/analyze the colours_code (warning level) by selecting a particular volcano.   
    * Number of reports for a particular volcano
    * Plot the flight level of the ashes observed and erupted (e.g. fl_obs_cld_L1, fl_eruption_L1)

In [ ]:
vaacDF = spark.read.json('vaac_total_line.json')
vaacDF.cache()

In [ ]:
vaacDF.printSchema()

Now we are going to create a new dataframe (df) by filtering out the rows which we dont have an issued date. And later from these rows, we are going to visualize the id of the volcano, colour and date values. 

In [ ]:
from pyspark.sql.functions import col, concat, to_timestamp
from pyspark.sql.types import TimestampType
df = (vaacDF 
        .where(col('issued_date').isNotNull())
        .select('v_id','name', 'colour_code', concat('issued_date', 'issued_time').alias('date'))
        .withColumn('date', to_timestamp('date', 'yyyyMMddHHmm').cast(TimestampType()).alias('date'))
     )
df.show()

Next, we are going to create another dataframe (codedf) by replacing red, orange, yellow and green  by  "4","3","2","1" respectively. 

In [ ]:
from pyspark.sql.types import IntegerType
codedf = (df
              .where(col('colour_code').isNotNull())
              .replace(['red', 'orange', 'yellow', 'green'], ["4", "3", "2", "1"], 'colour_code')
              .withColumn('colour_code', col('colour_code').cast(IntegerType()))
         )
codedf.show()

Now, we want to visualise all the different volcano ids (v_id) that we have in the codedf dataframe.

In [ ]:
ids = codedf.select('v_id').distinct().rdd.flatMap(lambda x: x).collect()
print(ids)

We are going to create a pandas dataframe (data) from the codedf dataframe. In this dataframe we will store the v_ids which are equal 252010 (so we select a particular volcano), and we will store the date and colour_code information. 

In [ ]:
import pandas
data = codedf.where('v_id == "252010"').select('date', 'colour_code').toPandas()
print(data)

Finally, we print the dataframe for visualzing the change of colours over the years. 

In [ ]:
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt

plt.ioff()

In [ ]:
plt.scatter(data['date'].tolist(), data['colour_code'].tolist())
plt.yticks(range(0,5), ('', 'green', 'yellow', 'orange', 'red') )

plt.show()

We can also analyse the frequency of reports of a particu

In [ ]:
frequency = vaacDF.filter('v_id == 283040').select('v_id','issued_date').groupBy('issued_date').count()
fd = frequency.toPandas()
fd

In [ ]:
ax = fd.plot.bar('issued_date', 'count')
ax.set_title('Frequency of vaac reports for 283040')
ax.legend(numpoints=1, loc='upper left')
ax.set_xlabel('Date')
ax.set_ylabel('Frequency', color='g')
plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
plt.show()